In [29]:
import cv2
import mediapipe as mp
import numpy as np
import pyautogui
import time
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

# 1. Open Camera

In [2]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    res, frame = cap.read()
    cv2.imshow("Camera", frame)
    
    if cv2.waitKey(10) & 0xff == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# 2. Make Pose Detection

In [6]:
cap = cv2.VideoCapture(0)
#Setup Mediapipe
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        res, frame = cap.read()
        
        #Recolor Image BGR to RGB because mediapipe is only accept RGB Image
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make detection from pose model mediapipe
        results = holistic.process(image)
        
        #Recolor from RGB to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #Rendering the detection result
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        
        cv2.imshow("Camera", image)

        if cv2.waitKey(10) & 0xff == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# 3. Extract Joint Coordinates

In [51]:
cap = cv2.VideoCapture(0)
#Setup Mediapipe
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        res, frame = cap.read()
        
        #Recolor Image BGR to RGB because mediapipe is only accept RGB Image
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make detection from pose model mediapipe
        results = holistic.process(image)
        
        #Recolor from RGB to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract Landmarks
        try:
            landmarks = results.right_hand_landmarks.landmark
  
        except:
            pass
        
        #Rendering the detection result
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
                
        cv2.imshow("Camera", image)

        if cv2.waitKey(10) & 0xff == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [12]:
results??

In [9]:
for lndmrk in mp_holistic.HandLandmark:
    print(lndmrk)

HandLandmark.WRIST
HandLandmark.THUMB_CMC
HandLandmark.THUMB_MCP
HandLandmark.THUMB_IP
HandLandmark.THUMB_TIP
HandLandmark.INDEX_FINGER_MCP
HandLandmark.INDEX_FINGER_PIP
HandLandmark.INDEX_FINGER_DIP
HandLandmark.INDEX_FINGER_TIP
HandLandmark.MIDDLE_FINGER_MCP
HandLandmark.MIDDLE_FINGER_PIP
HandLandmark.MIDDLE_FINGER_DIP
HandLandmark.MIDDLE_FINGER_TIP
HandLandmark.RING_FINGER_MCP
HandLandmark.RING_FINGER_PIP
HandLandmark.RING_FINGER_DIP
HandLandmark.RING_FINGER_TIP
HandLandmark.PINKY_MCP
HandLandmark.PINKY_PIP
HandLandmark.PINKY_DIP
HandLandmark.PINKY_TIP


In [15]:
landmarks[mp_holistic.HandLandmark.THUMB_CMC.value]

x: 0.2598624527454376
y: 0.7935397028923035
z: -0.008222446776926517

# 4. Calculate Angles

In [3]:
def calculate_angle(a,b,c):
    a = np.array(a) #First Point
    b = np.array(b) #Second Point
    c = np.array(c) #Third Point
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360-angle
    
    return angle

In [4]:
def calculate_distace_point(a,b):
    a = np.array(a)
    b = np.array(b)
    
    distance = np.linalg.norm(a-b)
    
    return distance

In [58]:
INDEX_FINGER_TIP = [landmarks[mp_holistic.HandLandmark.INDEX_FINGER_TIP.value].x, landmarks[mp_holistic.HandLandmark.INDEX_FINGER_TIP.value].y]
INDEX_FINGER_DIP = [landmarks[mp_holistic.HandLandmark.INDEX_FINGER_DIP.value].x, landmarks[mp_holistic.HandLandmark.INDEX_FINGER_DIP.value].y]
INDEX_FINGER_PIP = [landmarks[mp_holistic.HandLandmark.INDEX_FINGER_PIP.value].x, landmarks[mp_holistic.HandLandmark.INDEX_FINGER_PIP.value].y]

In [59]:
INDEX_FINGER_TIP, INDEX_FINGER_DIP, INDEX_FINGER_PIP

([0.24528715014457703, 0.333548903465271],
 [0.23803144693374634, 0.3952375054359436],
 [0.2268681824207306, 0.4771801233291626])

In [60]:
calculate_angle(INDEX_FINGER_PIP, INDEX_FINGER_DIP, INDEX_FINGER_TIP)

178.95040565855945

In [61]:
calculate_angle(INDEX_FINGER_PIP, INDEX_FINGER_DIP, INDEX_FINGER_TIP)

178.95040565855945

# 4. Curl Counter

In [60]:
cap = cv2.VideoCapture(0)

action = ""
#Setup Mediapipe
with mp_holistic.Holistic(min_detection_confidence=0.4, min_tracking_confidence=0.4) as holistic:
    while cap.isOpened():
        res, frame = cap.read()
        
        #Recolor Image BGR to RGB because mediapipe is only accept RGB Image
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make detection from pose model mediapipe
        results = holistic.process(image)
        
        #Recolor from RGB to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract Landmarks
        try:
            landmarks = results.right_hand_landmarks.landmark
            #landmarks2 = results.left_hand_landmarks.landmark
            #Get angle on right hand index finger
            INDEX_FINGER_TIP = [landmarks[mp_holistic.HandLandmark.INDEX_FINGER_TIP.value].x, landmarks[mp_holistic.HandLandmark.INDEX_FINGER_TIP.value].y]
            INDEX_FINGER_DIP = [landmarks[mp_holistic.HandLandmark.INDEX_FINGER_DIP.value].x, landmarks[mp_holistic.HandLandmark.INDEX_FINGER_DIP.value].y]
            INDEX_FINGER_PIP = [landmarks[mp_holistic.HandLandmark.INDEX_FINGER_PIP.value].x, landmarks[mp_holistic.HandLandmark.INDEX_FINGER_PIP.value].y]
            angle = calculate_angle(INDEX_FINGER_PIP, INDEX_FINGER_DIP, INDEX_FINGER_TIP)
            
            #Get angle on right hand thumb
            THUMB_FINGER_TIP = [landmarks[mp_holistic.HandLandmark.THUMB_TIP.value].x, landmarks[mp_holistic.HandLandmark.THUMB_TIP.value].y]
            THUMB_FINGER_IP = [landmarks[mp_holistic.HandLandmark.THUMB_IP.value].x, landmarks[mp_holistic.HandLandmark.THUMB_IP.value].y]
            THUMB_FINGER_MCP = [landmarks[mp_holistic.HandLandmark.THUMB_MCP.value].x, landmarks[mp_holistic.HandLandmark.THUMB_MCP.value].y]
            angle_t = calculate_angle(THUMB_FINGER_TIP, THUMB_FINGER_IP, THUMB_FINGER_MCP)
            
            #Visualize the angle
#             cv2.putText(image, str(angle_t),
#                        tuple(np.multiply(THUMB_FINGER_TIP, [640,480]).astype(int)),
#                             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
            
            #Get Distance between thumb ip and index mcp
            thumb_ip = [landmarks[mp_holistic.HandLandmark.THUMB_IP.value].x, landmarks[mp_holistic.HandLandmark.THUMB_IP.value].y]
            finger_tip = [landmarks[mp_holistic.HandLandmark.INDEX_FINGER_MCP.value].x, landmarks[mp_holistic.HandLandmark.INDEX_FINGER_MCP.value].y]
            distance = calculate_distace_point(thumb_ip, finger_tip)
            
            #Get Distance between thumb ip and index mcp
            middle_finger_tip = [landmarks[mp_holistic.HandLandmark.MIDDLE_FINGER_TIP.value].x, landmarks[mp_holistic.HandLandmark.MIDDLE_FINGER_TIP.value].y]
            finger_tip_2 = [landmarks[mp_holistic.HandLandmark.INDEX_FINGER_TIP.value].x, landmarks[mp_holistic.HandLandmark.INDEX_FINGER_TIP.value].y]
            distance_2 = calculate_distace_point(middle_finger_tip, finger_tip_2)
            
#             thumb_ip_l = [landmarks2[mp_holistic.HandLandmark.THUMB_IP.value].x, landmarks2[mp_holistic.HandLandmark.THUMB_IP.value].y]
#             finger_tip_l = [landmarks2[mp_holistic.HandLandmark.INDEX_FINGER_MCP.value].x, landmarks2[mp_holistic.HandLandmark.INDEX_FINGER_MCP.value].y]
#             distance_l = calculate_distace_point(thumb_ip_l, finger_tip_l)
            
            #Visualize the distance
#             cv2.putText(image, str(distance_2),
#                        tuple(np.multiply(middle_finger_tip, [640,480]).astype(int)),
#                             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
           
            
        
            if angle > 160 and distance < 0.1 and distance_2 < 0.15:
                action = "Hold"
                pyautogui.keyUp('d')
                pyautogui.keyUp('space')
                pyautogui.keyUp('a')
                #time.sleep(0.1)
                
            elif distance_2 > 0.15:
                action = "back"
                pyautogui.keyDown('a')
                    
            elif angle < 160 and distance < 0.1:
                action = "forward"
                pyautogui.keyDown('d')
                
                #time.sleep(0.1)
                
            elif (angle_t < 160.0):
                action = "hit"
                pyautogui.click() 
                
            elif (angle > 160 and distance > 0.1) or (angle < 160 and distance > 0.1) :
                action = "jump"
                pyautogui.keyDown('space')
                time.sleep(0.5)
                pyautogui.keyUp('space')
           
                
#             if angle_l > 160 and distance_l < 0.1:
#                 action = "Hold"
#                 pyautogui.keyUp('a')
#                 pyautogui.keyUp('s')
#                 #time.sleep(0.1)
#             elif angle_l < 160 and distance_l < 0.1:
#                 action = "back"
#                 pyautogui.keyDown('a')
#                 #time.sleep(0.1)
#             elif (angle_l > 160 and distance_l > 0.1) or (angle_l < 160 and distance_l > 0.1) :
#                 action = "down"
#                 pyautogui.keyDown('s')
#                 #time.sleep(0.1)
        except:
            pass
        
        #Setup status box
        cv2.putText(image, action,
                      (0,50),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        #Rendering the detection result
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
                
        cv2.imshow("Camera", image)

        if cv2.waitKey(10) & 0xff == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()